In [2]:
import os
import random
import pandas as pd
from typing import TypedDict, List, Literal
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from dotenv import load_dotenv
import re
import requests
from bs4 import BeautifulSoup

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Configuración inicial
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model="gpt-4", temperature=0)

# Generar df con escenarios medievales
df_scenarios = pd.DataFrame({
    "scenario": ["Un joven noble se encuentra en un bosque mágico", 
                 "Un aventurero se encuentra en una cueva misteriosa", 
                 "Un aprendiz de mago se encuentra en una torre encantada"]
})

class AgentState(TypedDict):
    escenario: str
    heroe: str
    caramelo: str
    cantidad_desafios: int
    max_desafios: int
    desafio: str
    respuesta: str
    story: List[str]
    desafios_resueltos: int
    puzzle_solution: str

GENERADOR_INTRO_PROMPT = """
Eres un narrador creativo. Crea una introducción cautivadora para una historia que incorpore:
1. Nombre del héroe: {hero_name}
2. Dulce favorito del héroe: {favorite_candy}
3. Escenario aleatorio: {random_scenario}

Tu introducción debe tener 3-5 oraciones, presentar al héroe, incorporar el dulce y establecer el escenario. 
Usa un lenguaje sencillo y amigable, apropiado para niños. Puedes usar emojis para hacer la historia más divertida.
"""

PROMPT_MAGO_BLANCO = """
Eres el Mago Blanco, un sabio guardián del bosque mágico. Tu tarea es poner a prueba al héroe {hero_name} con un {challenge_type}.

Si es un acertijo, crea uno simple y apropiado para niños.
Si es una pregunta matemática, crea una pregunta de aritmética simple (suma, resta, multiplicación o división) con números entre 1 y 20.

Presenta tu desafío en este formato exacto:
<desafio>
(Escribe aquí el desafío)
</desafio>

<solucion>
Solución: (Escribe aquí la respuesta correcta)
</solucion>

Asegúrate de incluir tanto la etiqueta de desafío como la de solución.
"""

def intro_node(state: AgentState) -> AgentState:
    nombre_heroe = input("¿Cuál es el nombre de tu héroe? ")
    caramelo_favorito = input("¿Cuál es tu dulce favorito? ")
    row_random = random.randint(0, len(df_scenarios)-1)
    
    random_scenario = df_scenarios["scenario"].iloc[row_random]
    
    state['heroe'] = nombre_heroe
    state['caramelo'] = caramelo_favorito
    state['escenario'] = random_scenario
    
    formatted_prompt = GENERADOR_INTRO_PROMPT.format(
        hero_name=nombre_heroe,
        favorite_candy=caramelo_favorito,
        random_scenario=random_scenario
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Genera la introducción de la historia.")
    ]
    response = model.invoke(messages)
    state['story'] = [response.content]
    print("\nHistoria generada:")
    print(state['story'][0])
    return state

def mago_blanco_node(state: AgentState) -> AgentState:
    print("\nEl Mago Blanco está preparando un desafío...")
    challenge_type = random.choice(["acertijo", "matemática"])
    
    messages = [
        SystemMessage(content=PROMPT_MAGO_BLANCO.format(hero_name=state['heroe'], challenge_type=challenge_type)),
        HumanMessage(content=f"Crea un {challenge_type} para este héroe.")
    ]
    response = model.invoke(messages)
    
    desafio_match = re.search(r'<desafio>(.*?)</desafio>', response.content, re.DOTALL)
    solucion_match = re.search(r'<solucion>(.*?)</solucion>', response.content, re.DOTALL)
    
    if desafio_match and solucion_match:
        state['desafio'] = desafio_match.group(1).strip()
        state['puzzle_solution'] = solucion_match.group(1).strip().split("Solución:")[-1].strip()

    else:
        print("No se pudo generar un desafío específico. Usando un desafío predeterminado.")
        state['desafio'] = "¿Cuánto es 2 + 2?"
        state['puzzle_solution'] = "4"
    
    print("\nDesafío del Mago Blanco:")
    print(state['desafio'])
    
    state['cantidad_desafios'] = state.get('cantidad_desafios', 0) + 1
    return state

def human_response_node(state: AgentState) -> AgentState:
    print(f"\nDesafío del Mago Blanco: {state['desafio']}")
    state['respuesta'] = input("¿Cuál es tu respuesta al desafío del Mago Blanco? ")
    print(f"Tu respuesta: {state['respuesta']}")
    return state

def puzzle_evaluator_node(state: AgentState) -> AgentState:
    puzzle_solution = state.get('puzzle_solution', 'N/A').lower()
    user_response = state['respuesta'].lower()

    # Comprobación directa para preguntas matemáticas
    if puzzle_solution.isdigit() and user_response.isdigit():
        is_correct = int(puzzle_solution) == int(user_response)
    else:
        # Para acertijos, usamos una comparación más flexible
        is_correct = any(word in puzzle_solution for word in user_response.split())

    if is_correct:
        explanation = f"¡Correcto! La respuesta '{state['respuesta']}' es acertada."
        state['desafio_resuelto'] = True
        state['desafios_resueltos'] = state.get('desafios_resueltos', 0) + 1
    else:
        explanation = f"Lo siento, la respuesta no es correcta. La solución era: {puzzle_solution}"
        state['desafio_resuelto'] = False

    print(f"\nEl Mago Blanco dice: {explanation}")
    return state

def should_continue(state: AgentState) -> Literal["continuar", "terminar"]:
    if state['desafios_resueltos'] >= 2:
        print("\n🎉 ¡Felicidades! Has superado los desafíos del Mago Blanco.")
        return "terminar"
    elif state['cantidad_desafios'] < state['max_desafios']:
        print("\n🔄 El Mago Blanco te dará otro desafío.")
        return "continuar"
    else:
        print("\n⏰ Has agotado tus oportunidades con el Mago Blanco.")
        return "terminar"

# Creación del grafo
builder = StateGraph(AgentState)

# Agregar nodos
builder.add_node("presentacion_historia", intro_node)
builder.add_node("mago_blanco", mago_blanco_node)
builder.add_node("respuesta_humano", human_response_node)
builder.add_node("evaluador_desafio", puzzle_evaluator_node)

# Definir bordes
builder.add_edge("presentacion_historia", "mago_blanco")
builder.add_edge("mago_blanco", "respuesta_humano")
builder.add_edge("respuesta_humano", "evaluador_desafio")

# Borde condicional basado en la evaluación del desafío
builder.add_conditional_edges(
    "evaluador_desafio",
    should_continue,
    {
        "continuar": "mago_blanco",
        "terminar": END
    }
)

# Establecer el punto de entrada
builder.set_entry_point("presentacion_historia")

# Compilar el grafo
graph = builder.compile()

# Ejecutar el grafo
state = {
    "max_desafios": 5,
    "cantidad_desafios": 0,
    "desafios_resueltos": 0,
    "story": [],
    "puzzle_solution": "N/A"
}

print("\n🧙‍♂️ ¡Bienvenido a la aventura en el bosque mágico! 🧙‍♂️")
try:
    final_state = graph.invoke(state)
    
    print("\n📜 Resumen final:")
    print(f"Historia: {final_state['story'][0]}")
    print(f"Desafíos enfrentados: {final_state['cantidad_desafios']}")
    print(f"Desafíos resueltos: {final_state['desafios_resueltos']}")
    if final_state['desafios_resueltos'] >= 2:
        print("🎊 El héroe ha superado los desafíos del Mago Blanco y continúa su aventura en el bosque mágico.")
    else:
        print("🔁 El héroe no ha superado todos los desafíos del Mago Blanco y debe buscar otro camino en el bosque.")
except Exception as e:
    print(f"Error durante la ejecución del juego: {e}")
    print("A pesar del error, la aventura continúa...")


🧙‍♂️ ¡Bienvenido a la aventura en el bosque mágico! 🧙‍♂️

Historia generada:
En una tierra llena de misterios y aventuras, vivía un valiente explorador llamado Fede 🧑‍🌾. Fede, con su mochila siempre llena de su dulce favorito, las titas 🍫, se encontraba en la entrada de una cueva misteriosa 🕸️. ¿Qué secretos descubrirá nuestro héroe en esta nueva aventura? ¡Vamos a descubrirlo juntos! 🌟

El Mago Blanco está preparando un desafío...

Desafío del Mago Blanco:
Fede, si tienes 15 manzanas y le das 7 a tu amigo, ¿cuántas manzanas te quedan?

Desafío del Mago Blanco: Fede, si tienes 15 manzanas y le das 7 a tu amigo, ¿cuántas manzanas te quedan?
Tu respuesta: 8

El Mago Blanco dice: ¡Correcto! La respuesta '8' es acertada.

🔄 El Mago Blanco te dará otro desafío.

El Mago Blanco está preparando un desafío...

Desafío del Mago Blanco:
Estoy lleno de llaves pero no puedo abrir ninguna puerta. ¿Qué soy?

Desafío del Mago Blanco: Estoy lleno de llaves pero no puedo abrir ninguna puerta. ¿Qué soy